# Preparing for Streamlit Deployment

## Tracking Files 

- Create a config/filepaths.json file.
- Define the following `filepaths` in your FPATHS dictionary/flepaths.json:
    - "data":
        - which should have a "full"/"ml"/"nn"
    - `...` **RETURN TO THIS LATER**, include: 'images','meta','?'

- **Produce a wordcloud** figure comparing the Low vs. High Ratings groups
    - Save the wordcloud figure to the filepath using your FPATHS dictionary.
- [Optional?] Produce a Scattertext Explorer comparing the two groups.
    -  and save as HTML using the filepath in your FPATHS dictionary.


## Example Code Snippets I'd Like to Add in an NLP Streamlit Lesson

In [1]:
## our pre-exting ngram df function https://login.codingdojo.com/m/692/16501/124388

import pandas as pd
def get_ngram_measures_finder(tokens, ngrams=2, get_scores_df=False, measure='raw_freq', top_n=None, min_freq = 1,
                             words_colname='Words'):
    import nltk
    if ngrams == 4:
        MeasuresClass = nltk.collocations.QuadgramAssocMeasures
        FinderClass = nltk.collocations.QuadgramCollocationFinder
        
    elif ngrams == 3: 
        MeasuresClass = nltk.collocations.TrigramAssocMeasures
        FinderClass = nltk.collocations.TrigramCollocationFinder
    else:
        MeasuresClass = nltk.collocations.BigramAssocMeasures
        FinderClass = nltk.collocations.BigramCollocationFinder

    measures = MeasuresClass()
    
   
    finder = FinderClass.from_words(tokens)
    finder.apply_freq_filter(min_freq)
    if measure=='pmi':
        scored_ngrams = finder.score_ngrams(measures.pmi)
    else:
        measure='raw_freq'
        scored_ngrams = finder.score_ngrams(measures.raw_freq)

    df_ngrams = pd.DataFrame(scored_ngrams, columns=[words_colname, measure.replace("_",' ').title()])
    if top_n is not None:
        return df_ngrams.head(top_n)
    else:
        return df_ngrams



In [ ]:
# Set the desired minimum frequency value, top-n and measure
ngram_kws = dict(measure='pmi',top_n =10 , min_freq=10)

# Get group bigrams
df_ngrams_high =  get_ngram_measures_finder(high_review_tokens_list,**ngram_kws,
                                                words_colname="ngrams (High Ratings)")
df_ngrams_low = get_ngram_measures_finder(low_review_tokens_list,**ngram_kws,
                                              words_colname= "ngrams (Low Ratings)")

# Concatenate and save top 10
df_compare_nrams = pd.concat(
    [df_ngrams_low,df_ngrams_high, ], axis=1,
    # keys=["High Rating Reviews", "Low Rating Reviews"],
)
df_compare_ngrams

Streamlit implementation:

In [ ]:
## NOTE: needs the measure added and freq_filter added (already in our function)

## Add creating ngrams
st.subheader('N-Grams')


ngrams = st.radio('n-grams', [2,3,4],horizontal=True,index=1)
top_n = st.selectbox('Compare Top # Ngrams',[10,15,20,25],index=2)

## Test Code for Making New Trigrams for both groups

@st.cache_data
def show_group_ngrams(GROUPS, col='tokens_nostop-combined-list', ngrams=3, top_n=20):
	ngram_kws = dict(ngrams=ngrams,  get_scores_df=True, top_n=top_n)
	
	df_ngrams_1star = get_ngram_measures_finder(tokens=GROUPS[1][col], words_colname=f"1 Star n-grams", **ngram_kws)
	df_ngrams_1star.columns =[ "1 Star n-grams", "1 Star-Freq"]
							  
	df_ngrams_5star= get_ngram_measures_finder(tokens=GROUPS[5][col], words_colname=f"5 Star n-grams", **ngram_kws)
	df_ngrams_5star.columns =[ "5 Star n-grams", "5 Star-Freq"]
	
	combined_ngrams = pd.concat([df_ngrams_1star,df_ngrams_5star],axis=1)
	return combined_ngrams

ngrams_df = show_group_ngrams(GROUPS=GROUPS,ngrams=ngrams,top_n=top_n)
st.dataframe(ngrams_df, hide_index=True, width=1000, height=None)

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=y.unique())
explanation = explainer.explain_instance(X_to_pred, clf_bayes.predict_proba)
explanation.show_in_notebook()